# 02 — Control: Governance & Security

This notebook demonstrates **AI Readiness Pillar #2: Control**.

You will:
- Assign dataset ownership
- Implement a basic **policy check** (who can see what)
- Record **audit logs** for data access
- Apply a simple row-level filter (department-based)

> In production you would use Unity Catalog / Purview / policy engines — here we model the concepts.


## 📺 Related Video

This notebook corresponds to:

**AI Readiness Explained — What “AI-Ready” Really Means (Part 1)**  
▶️ https://youtu.be/DfG2y-90-wc

If you prefer to understand the *architecture and design reasoning* before running the code, watch the video first.


In [ ]:
from pathlib import Path
import pandas as pd
from datetime import datetime
from src.policies import UserContext, apply_row_level_policy

BASE_PATH = Path("..")
DATA_CURATED = BASE_PATH / "data" / "curated"
AUDIT_PATH = BASE_PATH / "data" / "curated" / "audit_logs.csv"


In [ ]:
customers = pd.read_csv(DATA_CURATED / "customers_silver.csv")
orders = pd.read_csv(DATA_CURATED / "orders_silver.csv")

customers.head()


## Ownership registry

Governance begins with **accountability**.
Create an explicit mapping:
- dataset → owner / steward
- sensitivity level
- allowed roles


In [ ]:
ownership_registry = pd.DataFrame([
    {"dataset":"customers_silver", "owner":"data.steward@org", "sensitivity":"confidential", "allowed_roles":"steward,analyst"},
    {"dataset":"orders_silver", "owner":"finance.steward@org", "sensitivity":"internal", "allowed_roles":"steward,analyst,viewer"},
])
ownership_registry


## Policy-based access (simple demo)

We'll simulate three user contexts and enforce:
- role checks
- row-level filtering by department


In [ ]:
def can_access(dataset: str, ctx: UserContext) -> bool:
    row = ownership_registry[ownership_registry["dataset"]==dataset].iloc[0]
    allowed = [r.strip() for r in row["allowed_roles"].split(",")]
    return ctx.role in allowed

def audit(event: dict):
    df = pd.DataFrame([event])
    if AUDIT_PATH.exists():
        df_all = pd.read_csv(AUDIT_PATH)
        df_all = pd.concat([df_all, df], ignore_index=True)
    else:
        df_all = df
    df_all.to_csv(AUDIT_PATH, index=False)

users = [
    UserContext(user="alice", role="analyst", allowed_departments=["Finance"]),
    UserContext(user="bob", role="viewer", allowed_departments=["Admissions"]),
    UserContext(user="carol", role="steward", allowed_departments=["Finance","HR","Admissions","Research"]),
]

users


In [ ]:
dataset = "customers_silver"

for u in users:
    if not can_access(dataset, u):
        print(f"DENY: {u.user} ({u.role}) cannot access {dataset}")
        audit({"ts": datetime.utcnow().isoformat(), "user": u.user, "role": u.role, "dataset": dataset, "action":"DENY"})
        continue

    scoped = apply_row_level_policy(customers, u, "department")
    print(f"ALLOW: {u.user} ({u.role}) rows={len(scoped)} departments={u.allowed_departments}")
    audit({"ts": datetime.utcnow().isoformat(), "user": u.user, "role": u.role, "dataset": dataset, "action":"ALLOW", "rows": len(scoped)})


In [ ]:
pd.read_csv(AUDIT_PATH).tail(10)


## Why this matters for AI

If an AI system can retrieve or act on data that a user is not allowed to see, you have:
- compliance risk
- data leakage
- reputational damage

**Control** ensures your AI system inherits enterprise permissions.
